# Loading Data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving TrainingWiDS2021.csv to TrainingWiDS2021.csv


In [ ]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
dat = pd.read_csv(io.BytesIO(uploaded['TrainingWiDS2021.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
print(dat.shape)

(130157, 181)


In [ ]:
dat.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,...,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0,0,0,0,0,0,0,1
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,93,admit,Med-Surg ICU,0.000694,0,95.3,NaN,122.0,703.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0,68.0,NaN,NaN,NaN,NaN,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,92,admit,CTICU,0.000694,0,61.7,NaN,203.0,1206.03,1,0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1,60.0,30.0,30.0,142.0,7.39,4.0,...,11.6,11.6,34.0,34.0,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0,0,0,0,0,0,0,0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,91,admit,Med-Surg ICU,0.073611,0,NaN,NaN,119.0,601.01,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0,103.0,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [ ]:
y=dat['diabetes_mellitus']

In [ ]:
y.value_counts()

0    102006
1     28151
Name: diabetes_mellitus, dtype: int64

The classification variable is skewed. Use stratified split??

In [ ]:
#drop id, y
dat = dat.drop(columns=['Unnamed: 0','encounter_id','hospital_id', 'icu_id','diabetes_mellitus'])

dat.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('O')], dtype=object)

# EDA / Cleaning Data

Handling Missing Values

For categorical/binary values - use the most common class to impute

For numerical values* - use the mean to impute

*when the variable is majority missing, do not use the mean. Covert the NaN to a value that is out of range (-999).


## Categorial variables

In [ ]:
# dataset of categorical variables
cat = dat.select_dtypes(include=['O'])

In [ ]:
cat.head(25)

,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type
0,Caucasian,M,Floor,Floor,admit,CTICU
1,Caucasian,F,Floor,Floor,admit,Med-Surg ICU
2,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
3,Caucasian,F,Operating Room,Operating Room / Recovery,admit,CTICU
4,Caucasian,M,NaN,Accident & Emergency,admit,Med-Surg ICU
5,Caucasian,M,Direct Admit,Accident & Emergency,admit,Med-Surg ICU
6,Caucasian,F,Operating Room,Accident & Emergency,admit,Med-Surg ICU
7,Caucasian,M,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
8,Caucasian,M,Other Hospital,Other Hospital,admit,CCU-CTICU
9,NaN,M,Direct Admit,Accident & Emergency,admit,CCU-CTICU


In [ ]:
## function to check % of missing variables in each column
# dat = data set to check
# prct_val = returns columns missing more than this % (0 - 100)

def check_prct_miss(dat, prct_val):
  prct_miss = round(dat.isnull().mean()*100,2) 
  out = prct_miss[prct_miss > prct_val]

  return(out)

In [ ]:
check_prct_miss(cat, 0)

ethnicity                 1.22
gender                    0.05
hospital_admit_source    25.51
icu_admit_source          0.18
dtype: float64

In [ ]:
# binary variables: treat as categorical and NOT as numeric
binary = ["elective_surgery", "readmission_status", "apache_post_operative", "arf_apache", "gcs_unable_apache", "intubated_apache", "ventilated_apache",
"aids", "cirrhosis", "hepatic_failure", "immunosuppression", "leukemia", "lymphoma", "solid_tumor_with_metastasis"]

bin = dat[binary]
bin.head()

,elective_surgery,readmission_status,apache_post_operative,arf_apache,gcs_unable_apache,intubated_apache,ventilated_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0.0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0.0,1,1,0,0,0,0,0,0,0
4,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0


In [ ]:
# these binary variables have numeric data types
bin.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [ ]:
check_prct_miss(bin, 0)

gcs_unable_apache    0.54
dtype: float64

only 1 binary variable with missing values (gcs_unable_apache). 4 cat variables with missing values (dist missing < 25% so safe to impute this values with the most common value).


In [ ]:
#example. most common class of hospital admit source to use as imputed value
cat['hospital_admit_source'].mode().loc[0]

'Emergency Department'

In [ ]:
## function to impute missing cat/bin variables with most common value
# dat = dataset to use

def fillna_cat(dat):
    for i in dat.columns:
      impute_val = dat[i].mode().loc[0]
      dat.loc[:,(i)] = dat[i].fillna(impute_val)
      
    return(dat)

In [ ]:
cat_imputed = fillna_cat(cat)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [ ]:
fillna_cat(cat).head(25)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type
0,Caucasian,M,Floor,Floor,admit,CTICU
1,Caucasian,F,Floor,Floor,admit,Med-Surg ICU
2,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
3,Caucasian,F,Operating Room,Operating Room / Recovery,admit,CTICU
4,Caucasian,M,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
5,Caucasian,M,Direct Admit,Accident & Emergency,admit,Med-Surg ICU
6,Caucasian,F,Operating Room,Accident & Emergency,admit,Med-Surg ICU
7,Caucasian,M,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
8,Caucasian,M,Other Hospital,Other Hospital,admit,CCU-CTICU
9,Caucasian,M,Direct Admit,Accident & Emergency,admit,CCU-CTICU


In [ ]:
bin_imputed = fillna_cat(bin)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
fillna_cat(bin).head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,elective_surgery,readmission_status,apache_post_operative,arf_apache,gcs_unable_apache,intubated_apache,ventilated_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0.0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0.0,1,1,0,0,0,0,0,0,0
4,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0


In [ ]:
#confirm no longer missing any values in cat vars
print(check_prct_miss(bin, 0))
print(check_prct_miss(cat, 0))

Series([], dtype: float64)
Series([], dtype: float64)


## Numeric variables

Missing values & imputation for remaining vars (numeric)

In [ ]:
# column names of already imputed cat/bin variables
cat_col_names = list(cat.columns.values) + list(bin.columns.values)

In [ ]:
cat_col_names

['ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type',
 'elective_surgery',
 'readmission_status',
 'apache_post_operative',
 'arf_apache',
 'gcs_unable_apache',
 'intubated_apache',
 'ventilated_apache',
 'aids',
 'cirrhosis',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis']

In [ ]:
# dataset of numeric values
num = dat.drop(columns=cat_col_names)

In [ ]:
num.head()

,age,bmi,height,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,...,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min
0,68.0,22.732803,180.3,0.541667,73.9,2.3,113.0,502.01,0.4,31.0,2.51,NaN,3.0,6.0,4.0,168.0,118.0,27.4,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,77.0,27.421875,160.0,0.927778,70.2,NaN,108.0,203.01,NaN,9.0,0.56,1.0,1.0,3.0,1.0,145.0,120.0,36.9,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,120.0,38.0,...,8.6,8.6,0.56,0.56,145.0,143.0,27.0,27.0,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0
2,25.0,31.952749,172.7,0.000694,95.3,NaN,122.0,703.03,NaN,NaN,NaN,NaN,3.0,6.0,5.0,NaN,102.0,NaN,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,NaN,NaN,NaN,NaN,88.0,48.0,88.0,48.0,96.0,68.0,NaN,NaN,102.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81.0,22.635548,165.1,0.000694,61.7,NaN,203.0,1206.03,NaN,NaN,NaN,0.6,4.0,6.0,5.0,185.0,114.0,25.9,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,NaN,8.0,62.0,30.0,48.0,42.0,48.0,42.0,116.0,92.0,92.0,52.0,84.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.6,11.6,34.0,34.0,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0
4,19.0,NaN,188.0,0.073611,NaN,NaN,119.0,601.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,NaN,NaN,NaN,NaN,99.0,57.0,99.0,57.0,89.0,60.0,NaN,NaN,104.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
num.dtypes.unique()

array([dtype('float64')], dtype=object)

In [ ]:
## total missing values per numeric variable
for column in num:
  print(column, num[column].isnull().sum())

age 4988
bmi 4490
height 2077
pre_icu_los_days 0
weight 3463
albumin_apache 78163
apache_2_diagnosis 1685
apache_3j_diagnosis 865
bilirubin_apache 82560
bun_apache 25411
creatinine_apache 24882
fio2_apache 99720
gcs_eyes_apache 2190
gcs_motor_apache 2190
gcs_verbal_apache 2190
glucose_apache 14696
heart_rate_apache 309
hematocrit_apache 26758
map_apache 420
paco2_apache 99720
paco2_for_ph_apache 99720
pao2_apache 99720
ph_apache 99720
resprate_apache 808
sodium_apache 24519
temp_apache 6611
urineoutput_apache 63167
wbc_apache 29475
d1_diasbp_invasive_max 95068
d1_diasbp_invasive_min 95068
d1_diasbp_max 277
d1_diasbp_min 277
d1_diasbp_noninvasive_max 1636
d1_diasbp_noninvasive_min 1636
d1_heartrate_max 262
d1_heartrate_min 262
d1_mbp_invasive_max 94868
d1_mbp_invasive_min 94868
d1_mbp_max 327
d1_mbp_min 327
d1_mbp_noninvasive_max 2228
d1_mbp_noninvasive_min 2228
d1_resprate_max 683
d1_resprate_min 683
d1_spo2_max 532
d1_spo2_min 532
d1_sysbp_invasive_max 95038
d1_sysbp_invasive_min 9503

the max and min variables have the same # of missing of values. so either has both values or none. can combine the min and max into avg?

In [ ]:
#check missing value dist in numeric variables
print(check_prct_miss(num, 50))

albumin_apache          60.05
bilirubin_apache        63.43
fio2_apache             76.62
paco2_apache            76.62
paco2_for_ph_apache     76.62
                        ...  
h1_arterial_ph_min      82.86
h1_arterial_po2_max     82.55
h1_arterial_po2_min     82.55
h1_pao2fio2ratio_max    87.12
h1_pao2fio2ratio_min    87.12
Length: 73, dtype: float64


In [ ]:
# list of the column names to exclude from imputing mean
num_imp_exclude = list((check_prct_miss(num, 50)).index)

In [ ]:
## function for imputing the numeric values:
# dat = dataset to use
# col_list = list of columns not using mean imputation


def fillna_num(dat, col_list):
    for i in dat.columns:
        
        if i in col_list:
        
            #fill max,min variables
            dat.loc[:,(i)].fillna(-999,inplace = True)
                        
        else:
            dat.loc[:,(i)] = dat[i].fillna(dat[i].mean())
            
            
    return(dat)

In [ ]:
num.head()

,age,bmi,height,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,...,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min
0,68.0,22.732803,180.3,0.541667,73.9,2.3,113.0,502.01,0.4,31.0,2.51,NaN,3.0,6.0,4.0,168.0,118.0,27.4,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,77.0,27.421875,160.0,0.927778,70.2,NaN,108.0,203.01,NaN,9.0,0.56,1.0,1.0,3.0,1.0,145.0,120.0,36.9,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,120.0,38.0,...,8.6,8.6,0.56,0.56,145.0,143.0,27.0,27.0,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0
2,25.0,31.952749,172.7,0.000694,95.3,NaN,122.0,703.03,NaN,NaN,NaN,NaN,3.0,6.0,5.0,NaN,102.0,NaN,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,NaN,NaN,NaN,NaN,88.0,48.0,88.0,48.0,96.0,68.0,NaN,NaN,102.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81.0,22.635548,165.1,0.000694,61.7,NaN,203.0,1206.03,NaN,NaN,NaN,0.6,4.0,6.0,5.0,185.0,114.0,25.9,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,NaN,8.0,62.0,30.0,48.0,42.0,48.0,42.0,116.0,92.0,92.0,52.0,84.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.6,11.6,34.0,34.0,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0
4,19.0,NaN,188.0,0.073611,NaN,NaN,119.0,601.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,NaN,NaN,NaN,NaN,99.0,57.0,99.0,57.0,89.0,60.0,NaN,NaN,104.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
num_imputed = fillna_num(num,num_imp_exclude)

In [ ]:
num_imputed.head()

,age,bmi,height,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,...,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min
0,68.0,22.732803,180.3,0.541667,73.900000,2.3,113.0,502.01,0.4,31.00000,2.510000,-999.0,3.00000,6.000000,4.000000,168.000000,118.0,27.400000,40.0,-999.0,-999.0,-999.0,-999.00,36.0,134.00000,39.3,1800.803417,14.100000,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,...,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.000000,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0
1,77.0,27.421875,160.0,0.927778,70.200000,-999.0,108.0,203.01,-999.0,9.00000,0.560000,1.0,1.00000,3.000000,1.000000,145.000000,120.0,36.900000,46.0,37.0,37.0,51.0,7.45,33.0,145.00000,35.1,1800.803417,12.700000,-999.0,-999.0,95.0,31.0,95.0,31.0,118.0,72.0,-999.0,-999.0,120.0,38.0,...,8.6,8.6,0.56,0.56,145.0,143.0,27.0,27.0,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0
2,25.0,31.952749,172.7,0.000694,95.300000,-999.0,122.0,703.03,-999.0,25.71807,1.481629,-999.0,3.00000,6.000000,5.000000,160.141416,102.0,32.975817,68.0,-999.0,-999.0,-999.0,-999.00,37.0,137.94526,36.7,1800.803417,12.187662,-999.0,-999.0,88.0,48.0,88.0,48.0,96.0,68.0,-999.0,-999.0,102.0,68.0,...,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.000000,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0
3,81.0,22.635548,165.1,0.000694,61.700000,-999.0,203.0,1206.03,-999.0,25.71807,1.481629,0.6,4.00000,6.000000,5.000000,185.000000,114.0,25.900000,60.0,30.0,30.0,142.0,7.39,4.0,137.94526,34.8,1800.803417,8.000000,62.0,30.0,48.0,42.0,48.0,42.0,116.0,92.0,92.0,52.0,84.0,84.0,...,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,11.6,11.6,34.0,34.0,1.6,1.1,-999.0,-999.0,43.0,43.0,-999.0,-999.0,-999.0,-999.0,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0
4,19.0,29.110260,188.0,0.073611,83.791104,-999.0,119.0,601.01,-999.0,25.71807,1.481629,-999.0,3.48829,5.484828,4.030203,160.141416,60.0,32.975817,103.0,-999.0,-999.0,-999.0,-999.00,16.0,137.94526,36.7,1800.803417,12.187662,-999.0,-999.0,99.0,57.0,99.0,57.0,89.0,60.0,-999.0,-999.0,104.0,90.0,...,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.000000,-999.

# Imputation (All)

I looked at the numeric and categorical variables separately just to help me figure out what to do with them. You can just use the functions/code in this section to directly clean the primary data set

In [ ]:
## function to check % of missing variables in each column
# dat = dataset to check
# prct_val = returns columns missing more than this % (0 - 100)

def check_prct_miss(dat, prct_val):
  prct_miss = round(dat.isnull().mean()*100,2) 
  out = prct_miss[prct_miss > prct_val]

  return(out)

In [ ]:
### function for imputing all types of variables together ###
# dat = dataset to use

def fillna_all(dat):
  # dataset of categorical variables
  cat = dat.select_dtypes(include=['O'])

  # binary variables, also treat as categorical
  binary = ["elective_surgery", "readmission_status", "apache_post_operative", "arf_apache", "gcs_unable_apache", "intubated_apache", "ventilated_apache", 
            "aids", "cirrhosis", "hepatic_failure", "immunosuppression", "leukemia", "lymphoma", "solid_tumor_with_metastasis"]

  #combined list of cat/bin variable names
  cat_col_names = list(cat.columns.values) + binary

  # dataset of numeric values
  num = dat.drop(columns=cat_col_names)


  # list of the column names to exclude from imputing mean (at least 50% of the values are missing per var)
  num_imp_exclude = list((check_prct_miss(num, 50)).index)

  ## imputing missing values
  for i in dat.columns:

    # for cat/bin variables fill missing values with most common class
    if i in cat_col_names:
      impute_val = dat[i].mode().loc[0]
      dat.loc[:,(i)] = dat[i].fillna(impute_val)

    #imputing numeric variables
    else:
      if i in num_imp_exclude:
        #fill variables mostly missing with value out of range
        dat.loc[:,(i)].fillna(-999,inplace = True)

        # fill all other numeric var with mean               
      else:
        dat.loc[:,(i)] = dat[i].fillna(dat[i].mean())

  return(dat)
            



In [ ]:
## dataset clean with imputation
dat_imputed = fillna_all(dat)
dat_imputed.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,...,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,admit,CTICU,0.541667,0,73.900000,2.3,113.0,502.01,0,0,0.4,31.00000,2.510000,-999.0,3.00000,6.000000,0.0,4.000000,168.000000,118.0,27.400000,0,40.0,-999.0,-999.0,-999.0,-999.00,36.0,134.00000,39.3,1800.803417,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.000000,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,0,0,0,0,0,0,0
1,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,admit,Med-Surg ICU,0.927778,0,70.200000,-999.0,108.0,203.01,0,0,-999.0,9.00000,0.560000,1.0,1.00000,3.000000,0.0,1.000000,145.000000,120.0,36.900000,0,46.0,37.0,37.0,51.0,7.45,33.0,145.00000,35.1,1800.803417,1,...,27.0,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0,0,0,0,0,0,0
2,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.000694,0,95.300000,-999.0,122.0,703.03,0,0,-999.0,25.71807,1.481629,-999.0,3.00000,6.000000,0.0,5.000000,160.141416,102.0,32.975817,0,68.0,-999.0,-999.0,-999.0,-999.00,37.0,137.94526,36.7,1800.803417,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.000000,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,0,0,0,0,0,0,0
3,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,admit,CTICU,0.000694,0,61.700000,-999.0,203.0,1206.03,1,0,-999.0,25.71807,1.481629,0.6,4.00000,6.000000,0.0,5.000000,185.000000,114.0,25.900000,1,60.0,30.0,30.0,142.0,7.39,4.0,137.94526,34.8,1800.803417,1,...,-999.0,11.6,11.6,34.0,34.0,1.6,1.1,-999.0,-999.0,43.0,43.0,-999.0,-999.0,-999.0,-999.0,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0,0,0,0,0,0,0
4,19.0,29.110260,0,Caucasian,M,188.0,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.073611,0,83.791104,-999.0,119.0,601.01,0,0,-999.0,25.71807,1.481629,-999.0,3.48829,5.484828,0.0,4.030203,160.141416,60.0,32.975817,0,103.0,-999.0,-999.0,-999.0,-999.00,16.0,137.94526,36.7,1800.803417,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.000000,-999.0,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,0,0,0,0,0,0,0


In [ ]:
#check there are no more missing values in this imputed dataset
check_prct_miss(dat_imputed, 0)

Series([], dtype: float64)

Idea: since the min/max variables are either both missing or both report and these have a lot of missing values anyways, combine the two columns into 1 as an avg

In [ ]:
### function to combine the min/max variables into one ###
# dat = data set to use (will want to use the already clean set with imputation)

def combo_min_max_var(dat):

  # look for the min/max variables
  search_values  = ["_min", "_max"]

  # dataset just of these variables
  min_max_dat = pd.DataFrame(dat.loc[:,dat.columns.str.contains('|'.join(search_values ))])

  # separate only to min and only to max
  min_dat = min_max_dat.loc[:,min_max_dat.columns.str.contains("_min")]
  max_dat = min_max_dat.loc[:,min_max_dat.columns.str.contains("_max")]

  #reconfigure the column names
  min_max_col_names = list(min_dat.columns.values)
  avg_col_names = [sub.replace('_min', '_avg') for sub in min_max_col_names] 

  # calculate the avg of each min/max variable pair
  avg_dat_list = [[] for i in range(len(min_dat.columns))]
  for i in range(len(min_dat.columns)):
    new_col = ((min_dat.iloc[:,i] + max_dat.iloc[:,i] )/ 2)
    new_col_list = new_col.to_list()
    avg_dat_list[i] = new_col_list


  #convert to dataframe
  avg_df = pd.DataFrame(avg_dat_list).transpose()
  avg_df.columns = avg_col_names

  #drop the orig min/max vars from dataset
  dat = dat.drop(columns=list(min_max_dat.columns.values))

  #add the new avg variables onto the data
  result = pd.merge(dat, avg_df, left_index=True, right_index=True, how="outer") 

  return(result)


In [ ]:
# use already imputed data to combine the min/max variables
dat_imputed_avg = combo_min_max_var(dat_imputed)
dat_imputed_avg.shape

(130157, 112)

In [ ]:
dat_imputed_avg.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,...,h1_sysbp_noninvasive_avg,h1_temp_avg,d1_albumin_avg,d1_bilirubin_avg,d1_bun_avg,d1_calcium_avg,d1_creatinine_avg,d1_glucose_avg,d1_hco3_avg,d1_hemaglobin_avg,d1_hematocrit_avg,d1_inr_avg,d1_lactate_avg,d1_platelets_avg,d1_potassium_avg,d1_sodium_avg,d1_wbc_avg,h1_albumin_avg,h1_bilirubin_avg,h1_bun_avg,h1_calcium_avg,h1_creatinine_avg,h1_glucose_avg,h1_hco3_avg,h1_hemaglobin_avg,h1_hematocrit_avg,h1_inr_avg,h1_lactate_avg,h1_platelets_avg,h1_potassium_avg,h1_sodium_avg,h1_wbc_avg,d1_arterial_pco2_avg,d1_arterial_ph_avg,d1_arterial_po2_avg,d1_pao2fio2ratio_avg,h1_arterial_pco2_avg,h1_arterial_ph_avg,h1_arterial_po2_avg,h1_pao2fio2ratio_avg
0,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,admit,CTICU,0.541667,0,73.900000,2.3,113.0,502.01,0,0,0.4,31.00000,2.510000,-999.0,3.00000,6.000000,0.0,4.000000,168.000000,118.0,27.400000,0,40.0,-999.0,-999.0,-999.0,-999.00,36.0,134.00000,39.3,1800.803417,0,...,123.000000,38.500000,2.3,0.4,30.500000,7.950000,2.370000,138.500000,17.000000,8.900000,27.400000,-999.00,1.15,233.000000,3.700000,135.00000,14.10000,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.000000,-999.0,-999.000,-999.0,-999.0
1,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,admit,Med-Surg ICU,0.927778,0,70.200000,-999.0,108.0,203.01,0,0,-999.0,9.00000,0.560000,1.0,1.00000,3.000000,0.0,1.000000,145.000000,120.0,36.900000,0,46.0,37.0,37.0,51.0,7.45,33.0,145.00000,35.1,1800.803417,1,...,83.000000,36.300000,1.6,0.5,10.000000,8.300000,0.635000,136.500000,26.500000,11.200000,36.500000,1.30,3.50,522.000000,4.000000,145.00000,18.00000,-999.0,-999.0,9.0,8.6,0.56,144.0,27.0,11.3,36.9,1.30,3.5,557.0,4.2,145.0,12.7,37.0,7.45,51.0,52.900000,37.0,7.450,51.0,51.0
2,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.000694,0,95.300000,-999.0,122.0,703.03,0,0,-999.0,25.71807,1.481629,-999.0,3.00000,6.000000,0.0,5.000000,160.141416,102.0,32.975817,0,68.0,-999.0,-999.0,-999.0,-999.00,37.0,137.94526,36.7,1800.803417,0,...,136.000000,36.700000,-999.0,-999.0,24.566481,8.272228,1.428663,144.295056,23.822608,11.180651,33.747567,-999.00,-999.00,200.404226,4.091154,138.42086,11.91912,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.000000,-999.0,-999.000,-999.0,-999.0
3,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,admit,CTICU,0.000694,0,61.700000,-999.0,203.0,1206.03,1,0,-999.0,25.71807,1.481629,0.6,4.00000,6.000000,0.0,5.000000,185.000000,114.0,25.900000,1,60.0,30.0,30.0,142.0,7.39,4.0,137.94526,34.8,1800.803417,1,...,124.570687,35.200000,-999.0,-999.0,24.566481,8.272228,1.428663,136.500000,23.822608,10.250000,29.950000,1.35,-999.00,120.500000,4.250000,138.42086,8.50000,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,11.6,34.0,1.35,-999.0,43.0,-999.0,-999.0,8.8,32.0,7.39,219.5,289.583333,34.5,7.355,301.0,337.0
4,19.0,29.110260,0,Caucasian,M,188.0,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.073611,0,83.791104,-999.0,119.0,601.01,0,0,-999.0,25.71807,1.481629,-999.0,3.48829,5.484828,0.0,4.030203,160.141416,60.0,32.975817,0,103.0,-999.0,-999.0,-999.0,-999.00,16.0,137.94526,36.7,1800.803417,0,...,125.000000,36.668281,-999.0,-999.0,24.566481,8.272228,1.428663,144.295056,23.822608,11.180651,33.747567,-999.00,-999.00,20

# Encoding

In [ ]:
# number of levels for each of the categorical variables
for column in cat:
  print(column, cat[column].value_counts().count())

ethnicity 6
gender 2
hospital_admit_source 15
icu_admit_source 5
icu_stay_type 3
icu_type 8


In [ ]:
# dist of each of levels in the cat variables
for column in cat:
  print(column)
  print( cat[column].value_counts() )

ethnicity
Caucasian           100236
African American     13911
Other/Unknown         6261
Hispanic              5049
Asian                 2198
Native American        915
Name: ethnicity, dtype: int64
gender
M    70518
F    59573
Name: gender, dtype: int64
hospital_admit_source
Emergency Department    51258
Operating Room          13757
Floor                   11324
Direct Admit             8253
Recovery Room            4563
Other Hospital           2417
Acute Care/Floor         2097
Step-Down Unit (SDU)     1607
PACU                     1031
Other ICU                 283
Chest Pain Center         253
ICU to SDU                 62
ICU                        37
Observation                10
Other                       7
Name: hospital_admit_source, dtype: int64
icu_admit_source
Accident & Emergency         76846
Operating Room / Recovery    27436
Floor                        21459
Other Hospital                3464
Other ICU                      712
Name: icu_admit_source, dtype: int64

how do we want to handle encoding the cat variables? since the hospital_admit_source var has 15 levels one hot encoding will make it very sparse... but it is only 1 variable so that might be ok

other options for encoding?
https://www.kaggle.com/subinium/11-categorical-encoders-and-benchmark

# MODELING

In [ ]:
# convert to numpy array for cv
X=dat_imputed_avg.to_numpy()
X.shape

(130157, 112)

In [ ]:
X

array([[68.0, 22.73280282, 0, ..., -999.0, -999.0, -999.0],
       [77.0, 27.421875, 0, ..., 7.45, 51.0, 51.0],
       [25.0, 31.95274882, 0, ..., -999.0, -999.0, -999.0],
       ...,
       [73.0, 32.26537065, 0, ..., 7.231, 163.0, 163.0],
       [81.0, 24.40857925, 0, ..., -999.0, -999.0, -999.0],
       [35.0, 30.13235818, 0, ..., -999.0, -999.0, -999.0]], dtype=object)

In [ ]:
## dropping the cat/bin variables from data for now since they haven't been encoded yet
prepare_data = dat_imputed_avg.drop(columns=cat_col_names)
X=prepare_data.to_numpy()
X.shape

(130157, 92)

In [ ]:
#using cv to compare performance

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3, random_state=1)  #15 
skf.get_n_splits(X,y)

print(skf)

#save each of the folds
X_train_list, X_test_list, y_train_list, y_test_list = [], [], [],[]

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
 
    print(X_train.shape,y_train.shape)

    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


StratifiedKFold(n_splits=3, random_state=1, shuffle=False)
(86771, 92) (86771,)
(86771, 92) (86771,)
(86772, 92) (86772,)


In [ ]:
X_train_list[0]

array([[  76.        ,   30.13894917,  152.4       , ..., -999.        ,
        -999.        , -999.        ],
       [  66.        ,   31.31487889,  170.        , ..., -999.        ,
        -999.        , -999.        ],
       [  69.        ,   29.26681784,  157.5       , ..., -999.        ,
        -999.        , -999.        ],
       ...,
       [  73.        ,   32.26537065,  177.8       , ...,    7.231     ,
         163.        ,  163.        ],
       [  81.        ,   24.40857925,  185.4       , ..., -999.        ,
        -999.        , -999.        ],
       [  35.        ,   30.13235818,  182.9       , ..., -999.        ,
        -999.        , -999.        ]])

In [ ]:
X_train

array([[  68.        ,   22.73280282,  180.3       , ..., -999.        ,
        -999.        , -999.        ],
       [  77.        ,   27.421875  ,  160.        , ...,    7.45      ,
          51.        ,   51.        ],
       [  25.        ,   31.95274882,  172.7       , ..., -999.        ,
        -999.        , -999.        ],
       ...,
       [  71.        ,   27.54760074,  170.2       , ..., -999.        ,
        -999.        , -999.        ],
       [  47.        ,   46.23592931,  172.7       , ...,    7.32      ,
          74.2       ,  123.6666667 ],
       [  57.        ,   34.07203249,  170.2       , ..., -999.        ,
        -999.        , -999.        ]])

## Simple Decision Tree

In [ ]:
# decision tree (only one of the train/test sets)

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.metrics import roc_auc_score

decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(X_train, y_train)

roc_auc_score(y_test, decision_tree.predict_proba(X_test)[:, 1])


0.7275606246507069

In [ ]:
# decision tree - use cross validation to fit model

auc_folds=[]

for j in range(3):  
  decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
  decision_tree = decision_tree.fit(X_train_list[j], y_train_list[j])

  auc = roc_auc_score(y_test_list[j], decision_tree.predict_proba(X_test_list[j])[:, 1])
  
  auc_folds.append(auc)



print(auc_folds)
  


[0.7279918683510569, 0.7444380544061071, 0.7275606246507069]


would have to do some parameter tuning to try to make this better? or use a diff model type